# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298405254721                   -0.85    5.1
  2   -8.300187740783       -2.75       -1.25    1.2
  3   -8.300447323029       -3.59       -1.89    3.6
  4   -8.300462555718       -4.82       -2.75    1.6
  5   -8.300464258675       -5.77       -3.03    8.2
  6   -8.300464434247       -6.76       -3.18    1.0
  7   -8.300464538691       -6.98       -3.32    1.5
  8   -8.300464591527       -7.28       -3.45    2.1
  9   -8.300464632814       -7.38       -3.67    1.0
 10   -8.300464636520       -8.43       -3.76    1.0
 11   -8.300464642284       -8.24       -3.97    1.0
 12   -8.300464643939       -8.78       -4.44    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63714109069                   -0.71    5.5
  2   -16.64111585643       -2.40       -1.15    1.0
  3   -16.67891627516       -1.42       -1.74    4.0
  4   -16.67911152924       -3.71       -2.30    4.9
  5   -16.67920817585       -4.01       -2.72    4.4
  6   -16.67927773333       -4.16       -3.11    3.5
  7   -16.67928278131       -5.30       -3.28    2.6
  8   -16.67928611406       -5.48       -3.99    2.6
  9   -16.67928621855       -6.98       -4.46    2.6


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32684255169                   -0.56    7.0
  2   -33.33439492621       -2.12       -1.00    1.6
  3   -33.33594151490       -2.81       -1.72    7.5
  4   -33.33612975023       -3.73       -2.63    3.6
  5   -33.33676269544       -3.20       -2.20    6.1
  6   -33.33679502190       -4.49       -2.24    1.2
  7   -33.33694324300       -3.83       -3.29    4.6
  8   -33.33694388903       -6.19       -3.66    5.6
  9   -33.33694377677   +   -6.95       -3.74    1.8
 10   -33.33694391581       -6.86       -4.32    4.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298686312499                   -0.85    5.2
  2   -8.300294957389       -2.79       -1.59    1.0
  3   -8.300459892993       -3.78       -2.67    4.8
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│     100
│       3
│       3
│       3
│       3
│       2
│       5
│       2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:34
  4   -8.300433766394   +   -4.58       -2.51   15.1
  5   -8.300464511678       -4.51       -3.69    3.1
  6   -8.300464617000       -6.98       -4.10    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32693710867                   -0.56    7.5
  2   -33.33358381829       -2.18       -1.28    1.1
  3   -32.69293441932   +   -0.19       -1.15    8.5
  4   -32.28594491419   +   -0.39       -1.05    5.6
  5   -33.23721021131       -0.02       -1.34    4.9
  6   -32.81435345456   +   -0.37       -1.20    4.8
  7   -33.30558725747       -0.31       -1.76    5.1
  8   -33.33641360360       -1.51       -2.35    2.8
  9   -33.33690686163       -3.31       -2.82    2.8
 10   -33.33690214545   +   -5.33       -2.85    3.4
 11   -33.33692693397       -4.61       -3.09    1.8
 12   -33.33694317122       -4.79       -3.79    2.2
 13   -33.33694335813       -6.73       -3.90    4.2
 14   -33.33694340742       -7.31       -4.12    2.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.